<a href="https://colab.research.google.com/github/GenAIUnplugged/langchain_series/blob/main/03_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-openai langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=0,model="gpt-4o-mini")

# LLMChain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Define a prompt template
template = """You are a helpful assistant.
Answer the following question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Create the LLMChain
# We're assuming 'model' (a ChatOpenAI instance) has been defined previously
chain = LLMChain(prompt=prompt, llm=model)

# Invoke the chain with a question
question_text = "What is the capital of Bulgaria?"
response = chain.invoke({"question":question_text})

# Print the response
print(response)

{'question': 'What is the capital of Bulgaria?', 'text': 'The capital of Bulgaria is Sofia.'}


# SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

# Define the first chain (we already have `chain` from the previous example)

# Define a second prompt template and chain
template2 = """Given the capital city of a country, tell me something interesting about that city.
Capital city: {capital}
"""
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = LLMChain(prompt=prompt2, llm=model) # Use the same model

# Create the SimpleSequentialChain
# The output of the first chain becomes the input of the second chain
# The input variable names must match the output variable names
# In this case, the output of the first chain's default is 'text', which matches the input of the second chain's 'capital' if named 'text'
# However, if we want to explicitly map, we can define input_variables in chain2 to match chain's output.
# For SimpleSequentialChain, the output of the first chain is automatically passed as the single input to the second.
# We need to ensure the output key of the first chain matches the *single* input key of the second chain, or rely on the default.
# The default output key for an LLMChain is 'text'. So, if the second chain has a single input variable (like 'capital'),
# SimpleSequentialChain will automatically map the first chain's 'text' output to the second chain's 'capital' input.

sequential_chain = SimpleSequentialChain(chains=[chain, chain2], verbose=True)

# Run the sequential chain
# The input to the sequential chain is the input to the first chain ('question')
input_data = {"question": "What is the capital of Bulgaria?"}
response = sequential_chain.invoke(input_data)
print(response)



> Entering new SimpleSequentialChain chain...


ValueError: Missing some input keys: {'input'}

**Why we are getting error ?**\
Looking at the definition of the SimpleSequentialChain, it takes the output of the first chain and passes it as the single input to the second chain. By default, the LLMChain outputs its result under the key 'text'.

In this case, the first chain (chain) has an input key 'question'. The second chain (chain2) has an input key 'capital'. SimpleSequentialChain tries to map the output of the first chain to the input of the second chain. However, the error suggests that the SimpleSequentialChain itself is looking for a top-level input variable named 'input' before even running the first chain.

This is likely happening because SimpleSequentialChain expects the input it receives to be a single value (which it then passes to the first chain) or a dictionary with a single key named 'input' by default, unless the input key is explicitly defined. The current input {"question": "What is the capital of Bulgaria?"} does not match this expectation.

**Solution:**\
To resolve this, we need to ensure the input dictionary provided to sequential_chain.invoke() has the key that the SimpleSequentialChain is expecting. Based on the error message, this expected key is 'input'.

In [ ]:
input_data = {"input": "What is the capital of Bulgaria?"}
response = sequential_chain.invoke(input_data)
print(response)



> Entering new SimpleSequentialChain chain...
The capital of Bulgaria is Sofia.
Sofia, the capital of Bulgaria, is one of the oldest cities in Europe, with a history that dates back over 2,500 years. It was originally founded as a Thracian settlement called Serdica. An interesting fact about Sofia is that it is home to the Alexander Nevsky Cathedral, one of the largest Eastern Orthodox cathedrals in the world. This stunning architectural masterpiece, completed in 1912, features a striking gold-plated dome and is a symbol of Bulgarian national identity. The cathedral is not only a place of worship but also a significant cultural landmark, attracting visitors with its beautiful mosaics and impressive interior.

> Finished chain.
{'input': 'What is the capital of Bulgaria?', 'output': 'Sofia, the capital of Bulgaria, is one of the oldest cities in Europe, with a history that dates back over 2,500 years. It was originally founded as a Thracian settlement called Serdica. An interesting fa

# SequentialChain

In [ ]:
from google.colab import userdata
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

# Define the first chain: find the capital
template1 = """You are a helpful assistant.
Answer the following question: {question}
"""
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = LLMChain(llm=model, prompt=prompt1, output_key="capital_city")

# Define the second chain: tell something interesting about the capital
template2 = """Given the capital city of a country, tell me something interesting about that city.
Capital city: {capital_city}
"""
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = LLMChain(llm=model, prompt=prompt2, output_key="interesting_fact")

# Create the SequentialChain
# The input to the SequentialChain is the input to the first chain (question)
# The output of the first chain (capital_city) becomes an input to the second chain
# The final output of the SequentialChain includes inputs and outputs of the chains
sequential_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["question"],
    output_variables=["capital_city", "interesting_fact"],
    verbose=True
)

# Run the sequential chain
input_data = {"question": "What is the capital of Bulgaria?"}
response = sequential_chain.invoke(input_data)
response



> Entering new SequentialChain chain...

> Finished chain.


{'question': 'What is the capital of Bulgaria?',
 'capital_city': 'The capital of Bulgaria is Sofia.',
 'interesting_fact': 'Sofia, the capital of Bulgaria, is one of the oldest cities in Europe, with a history that spans over 2,000 years. It has been inhabited since at least the 8th century BC and has been known by various names throughout its history, including Serdica and Sredets. One interesting fact about Sofia is that it is located at the foot of Vitosha Mountain, which is a popular destination for hiking and skiing. The mountain is also home to the Vitosha Nature Park, offering a beautiful natural escape just a short distance from the urban environment. Additionally, Sofia is known for its rich cultural heritage, featuring a mix of Roman, Byzantine, and Ottoman influences, which can be seen in its architecture and numerous historical sites.'}

# RouterChain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `OpenAI`, or `Other`.
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": "how do I call GPT model?"})

'OpenAI'

In [ ]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

openai_chain = PromptTemplate.from_template(
    """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-4o-mini",temperature=0)


general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

This line creates the full_chain. Let's break down what's happening here:

{"topic": chain, "question": lambda x: x["question"]}: This part creates a dictionary of runnables.
"topic": chain: This maps the key "topic" to the chain object we defined previously. Recall that chain is responsible for classifying the input question into a topic ("LangChain", "OpenAI", or "Other"). When this part of the chain is executed, it will run the chain and the output will be associated with the key "topic".
"question": lambda x: x["question"]: This maps the key "question" to a lambda function. A lambda function is a small, anonymous function. In this case, it takes an input x (which will be the input dictionary passed to full_chain.invoke()) and returns the value associated with the key "question" from that input dictionary. This essentially just passes the original question through, making it available for later steps.
|: This is the pipe operator in LangChain, used to sequentially compose runnables. The output of the left side (the dictionary of runnables) is passed as input to the right side.
RunnableLambda(route): This wraps our route function within a RunnableLambda. The route function takes an input dictionary and determines which chain (anthropic_chain, langchain_chain, or general_chain) to execute next based on the "topic" key in the input dictionary. The RunnableLambda makes this function executable within the LangChain framework.
In essence, the full_chain first runs the chain to classify the input question's topic and simultaneously keeps the original question. Then, it uses the route function to select the appropriate downstream chain based on the classified topic.

In [ ]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content='To use Anthropic, you typically need to follow these steps:\n\n1. **Access the Platform**: Visit the Anthropic website or the specific platform where their AI models are hosted. You may need to create an account if you don’t have one.\n\n2. **Choose a Model**: Anthropic offers various AI models, such as Claude. Select the model that best fits your needs.\n\n3. **API Integration**: If you’re a developer, you can integrate Anthropic’s models into your applications using their API. Refer to the API documentation for details on how to authenticate and make requests.\n\n4. **Input Your Queries**: Whether using the web interface or API, input your queries or prompts. Be clear and specific to get the best responses from the model.\n\n5. **Review and Iterate**: Analyze the responses you receive. You may need to refine your prompts or queries to get more accurate or relevant results.\n\n6. **Follow Guidelines**: Ensure you adhere to any usage guidelines or ethical considerati

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route) \
full_chain.invoke({"question": "how do I use Anthropic?"}) \
This line executes the full_chain with the input {"question": "how do I use Anthropic?"}. The process will be:

1. The input {"question": "how do I use Anthropic?"} is passed to the first part of the chain {"topic": chain, "question": lambda x: x["question"]}.
2. The chain is executed with {"question": "how do I use Anthropic?"} as input, likely classifying it as "OpenAI" (since Anthropic is often associated with the same domain as OpenAI in these examples). The output of chain becomes the value for the "topic" key.
3. The lambda x: x["question"] is executed with the input {"question": "how do I use Anthropic?"}, and the value "how do I use Anthropic?" becomes the value for the "question" key.
4. The output of this first step is a dictionary like {"topic": "OpenAI", "question": "how do I use Anthropic?"}. This dictionary is passed as input to the RunnableLambda(route).
5. The route function is called with this dictionary. It checks the value of "topic" ("OpenAI"). Based on the route function's logic (specifically the elif "langchain" in info["topic"].lower(): return langchain_chain and the else: return general_chain parts, assuming "OpenAI" doesn't exactly match "anthropic" or "langchain"), it will likely select the general_chain.
6. The selected chain (general_chain in this case) is then executed with the original question "how do I use Anthropic?" as input.
7. The final response is the output of the chain selected by the router.

In [ ]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you should start by installing the library via pip. You can do this by running `pip install langchain`. Once installed, you can begin by importing the necessary modules and components based on your use case, such as language models, chains, or agents.\n\nNext, you can create a language model instance, configure it with your desired parameters, and then build a chain or agent that utilizes this model. LangChain provides various components to help you structure your application, including prompt templates, memory, and tools for integrating with external APIs or databases.\n\nFinally, you can run your chain or agent to process inputs and generate outputs, allowing you to leverage the power of language models in your applications. Be sure to check the official documentation for detailed examples and best practices to get the most out of LangChain.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'compl

In [ ]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 24, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BVgqwk4aV0rNR1LY68uJ0gbgmwz4a', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--60c35e96-01a7-4d29-95ef-81ef5c7db1d5-0', usage_metadata={'input_tokens': 24, 'output_tokens': 9, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})